In [5]:
import copy
import operator
import numpy as np
from random import randrange
from scipy.special import softmax
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from sklearn.preprocessing import minmax_scale,normalize
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from skmultiflow.data.stagger_generator import STAGGERGenerator

In [3]:
def get_data(nsample,classfun,rnd):
    stream = STAGGERGenerator(classification_function = classfun, random_state = rnd, balance_classes = False)
    stream.prepare_for_use()
    X,y = stream.next_sample(nsample)
    return X,y    

def CNE():
    """CNE create new expert"""
    clf = GaussianNB()
    return clf

def RE(w,e,t):
    """
    RE remove experts - return experts and weights
    with larger weights than theta
    """
    theta = t
    nw = w[w>theta]
    ne = e[w>theta]
    return nw,ne

#initial data###############

#X0,y0 = get_data(100,0,112)
#X_init,y_init = X0,y0

X0,y0 = get_data(33,0,112)
X1,y1 = get_data(33,1,112)
X2,y2 = get_data(34,2,112)
X = np.r_[X0,X1,X2]
y = np.r_[y0,y1,y2]
X_init = X
y_init = shuffle(y, random_state=0)

#train some initial classifier#######
e = np.empty(0)
e = np.append(e,copy.deepcopy(CNE()))
e[0].fit(X_init,y_init)
#DWM parameters######################
beta = 0.5
theta = 0.02
w = np.array([1.]) #list of weights
p = 1
#collect data###################
tlist = [] # time
sclist = [] # score
explist = [] # number of experts
################################

for t in range(0,120):
    score = 0
    if t < 40:
        classfun = 2
    elif t>=40 and t  <= 89:
        classfun = 0
    elif t>89:
        classfun = 1
        
    #batch of examples at time t
    X,y = get_data(100,classfun,randrange(0,1200))
    #loop over new examples
    for i in range(0,100-5):
        
        sigma = np.zeros(2) # 
        #loop over expert pool
        for j in range(len(e)):

            #lambda local, classify new example
            llocal = e[j].predict(X[i,:].reshape(1, -1))[0]

            if (llocal != y[i]) and (i%p == 0):
                w[j] = w[j]*beta

            sigma[int(llocal)] = sigma[int(llocal)] + w[j]

        index = np.argmax(sigma)
        lglobal = index  

        if (i%p == 0):

            w = softmax(w)
            #w = minmax_scale(w,feature_range=(1e-5,1))
            w,e = RE(w,e,theta)

            if (lglobal != y[i]):
                e = np.append(e,copy.deepcopy(CNE()))
                w = np.append(w,1.)

        #training of experts        
        for j in range(len(e)):
            e[j] = e[j].partial_fit(X[i:i+5,:],y[i:i+5],np.array([0,1]))

        if (lglobal==y[i]):
            score += 1


    #print(t,classfun,"score",score/95.,len(e),sigma,w)
    tlist.append(t);sclist.append(score/95.);explist.append(len(e))

In [4]:
plt.figure(figsize=(10,4))
plt.subplot(1, 2, 1)
plt.plot(tlist,sclist)
plt.xlabel("$t$"),plt.ylabel("accuracy")
plt.subplot(1, 2, 2)
plt.plot(tlist,explist)
plt.xlabel("$t$"),plt.ylabel("number of experts")

NameError: name 'plt' is not defined